In [1]:
#@title 1. Tải dữ liệu
train_folder = './data/cats_and_dogs_filtered/train'
valid_folder = './data/cats_and_dogs_filtered/validation'

!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

# Giải nén dữ liệu
main_folder = './data'
!unzip -q cats_and_dogs_filtered.zip -d $main_folder

--2023-03-15 14:27:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 142.250.99.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   188MB/s    in 0.3s    

2023-03-15 14:27:15 (188 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
# Sử dụng colab để review ảnh

In [3]:
#@title 2. Tải Pretrained model - Inception V3

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-03-15 14:27:16--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 142.250.99.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

./inception_v3_weig 100%[===================>]  83.84M   124MB/s    in 0.7s    

2023-03-15 14:27:17 (124 MB/s) - ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [4]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model


# 1. Sử dụng kiến trúc của mô hình
from tensorflow.keras.applications.inception_v3 import InceptionV3

# 2. Khởi tạo mô hình
#### TODO 1: Sử dụng mô hình InceptionV3
pre_trained_model = InceptionV3(  input_shape = (150, 150, 3),
                                                        include_top = False,
                                                        weights = None)                        


local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# 3. Load tham số đã được train vào mô hình
pre_trained_model.load_weights(local_weights_file)
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [5]:
pre_trained_model.trainable = False
#pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')

last_output = last_layer.output
last_output

<KerasTensor: shape=(None, 7, 7, 768) dtype=float32 (created by layer 'mixed7')>

In [6]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(1, activation='sigmoid')(x)           

# 11. Gắn mô hình phân loại của chúng ta vào pre trained model
model = Model( pre_trained_model.input, x) 

# 12. Bổ sung thuật toán trainining. RMSProp có khả năng điều chỉnh tốc độc học dựa vào độ lớn của đạo hàm

#### TODO 4: Compile thuật toán
model.compile(
    optimizer = RMSprop(lr = 0.0001),
    loss ='binary_crossentropy', #ham sai lệch
    metrics = ['acc'] # độ đo lường
)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 13. Bổ sung tăng cường data cho tập train
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# 13. Không bổ sung tăng cường data cho tập validation
validation_datagen = ImageDataGenerator( rescale = 1.0/255. )

# 14. Tiến hành load ảnh từ thư mục với tập train
train_generator = train_datagen.flow_from_directory(train_folder,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# 15. Tiến hành load ảnh từ thư mục với tập validation
validation_generator =  validation_datagen.flow_from_directory( valid_folder,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
callback = [EarlyStopping(monitor='val_accuracy', patience=3), ModelCheckpoint(filepath='mymodel.h5', monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=False, verbose=1)]
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 3, # số lần lặp qua toàn bộ tập dữ liệu
    validation_steps = 50 , 
    #batch : trong 1 lần train tập dữ liệu đó lại chia thành nhiều phần nhỏ để load
    verbose = 2 # in ra 
)

Epoch 1/3
100/100 - 33s - loss: 1.0579 - acc: 0.8330 - val_loss: 0.3907 - val_acc: 0.8760 - 33s/epoch - 334ms/step
Epoch 2/3
100/100 - 19s - loss: 0.2851 - acc: 0.9005 - val_loss: 0.1137 - val_acc: 0.9560 - 19s/epoch - 185ms/step
Epoch 3/3
100/100 - 21s - loss: 0.2215 - acc: 0.9170 - val_loss: 0.0856 - val_acc: 0.9700 - 21s/epoch - 209ms/step


# 6. Tải ảnh của bạn và tiến hành dự đoán

In [ ]:
#@title Tải ảnh
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
  image = plt.imread(fn)
  plt.imshow(image)
  

  image = Image.open(fn)
  # img = Image.fromarray(arr)
  image = image.resize(size=(150, 150))

  plt.imshow(image)
  data = np.asarray(image)
  data=np.expand_dims(data, 0)
  x = data / 255.0
  fi = model.predict(x)
  print(fi)
  print('Dog with output: {}'.format(fi[0])) if fi >= 0.5 else print('Cat with output: {}'.format(fi[0]))